# Importing Libraries

* System Append to set proper path

In [ ]:
sys.path.append('../')

* Default

In [ ]:
import lasio
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from collections import Counter

* Vaex

In [ ]:
import vaex

* Pandas Options

In [ ]:
pd.set_option('max_columns', None)

* Source Code

In [ ]:
from Source.Utils import welllog
from Source.Utils import multi_df
from Source.Utils import well_plot

* Tqdm Progress Bar

In [ ]:
%%capture
from tqdm import tqdm_notebook

# Reading Las files Dataset

In [ ]:
path = '../Data/GEOLINK_Lithology and wells NORTH SEA/'

npd_wells = welllog.read_las_directory(path)

* Las files Glance

In [ ]:
print('Number of Las Files read: ' + str(len(npd_wells)))
print('##########################')
print('Las files ID: ' + str(npd_wells.keys()))
print('##########################')
print(str(npd_wells['15_9-12'].curves))


        Note: The Mnmonic Table above does not necessarily represent all the available log curves on the dataset

# Main Dataframe Building and Processing

* Checking unmatching unit of measurement for each log curve

In [ ]:
unit_mismatch_list = welllog.unit_check(npd_wells)

* Converting all las files to dataframe

In [ ]:
npd_wells_df = {}

for id in tqdm_notebook(list(npd_wells.keys()), desc='Converting to dataframe'):

    npd_wells_df[id] = npd_wells[id].df()

        * Filling in Log Dataframes

In [ ]:
#logs_dict = welllog.log_frame(npd_wells_df, logs_list, mode='df')

* Creating Main Dataframe

        * Creating Well ID column

In [ ]:
for id in tqdm_notebook(list(npd_wells_df.keys()), desc='Adding Well Name Column'):

    npd_wells_df[id]['WELL_NAME'] = id

        * Converting Depth to a Column

In [ ]:
for id in tqdm_notebook(list(npd_wells_df.keys()), desc='Adding Depth Column'):

    npd_wells_df[id]['DEPTH'] = npd_wells_df[id].index

        * Selected Logs

In [ ]:
logs_list = ['WELL_NAME', 'DEPTH', 'CALI', 'NPHI', 'RHOB', 'GR', 'DTC', 'RDEP']


        * Creating Empty Dataframe

In [ ]:
df_main = pd.DataFrame(columns= logs_list)

        * Filling Dataframe

In [ ]:
for id in tqdm_notebook(list(npd_wells_df.keys()), desc='Adding Depth Column'):

    tmp = []

    for i in range(len(logs_list)):

        if logs_list[i] in npd_wells_df[id].columns:

            tmp.append(logs_list[i])

    df_main = df_main.append(npd_wells_df[id][tmp], ignore_index=True)     

In [ ]:
df_main.describe()

## Exploratory Data Analysis

* Correlation Matrix

In [ ]:
corrmat_df = abs(df_main.corr()) # absolute correlation

plt.figure(figsize=(15,10))

sns.heatmap(corrmat_df, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, cmap='Blues')

* Mixer Scatter Plot

In [ ]:
plt.figure(figsize=(15,10))

sns.set()

sns.pairplot(df_main.dropna().sample(1000))
plt.show();

* General Statistics

In [ ]:
df_main.describe()

* Distribution Information

        * CALI Variable 

In [ ]:
(mu, sigma) = stats.norm.fit(df_main[df_main.CALI.notnull()].CALI.values) 

In [ ]:
plt.figure(figsize=(15,10))

sns.distplot(df_main[df_main.CALI.notnull()].CALI.values); 

plt.legend('Normal distribution')
plt.ylabel('Frequency')
plt.title('CALI distribution -- Mu:' + ' ' + str(mu) + ' ' + 'Sigma:' + ' ' + str(sigma) )

        * NPHI Variable

In [ ]:
(mu, sigma) = stats.norm.fit(df_main[df_main.NPHI.notnull()].NPHI.values) 

In [ ]:
plt.figure(figsize=(15,10))

sns.distplot(df_main[df_main.NPHI.notnull()].NPHI.values);
#plt.xlim(0,100) 

plt.legend('Normal distribution')
plt.ylabel('Frequency')
plt.title('NPHI distribution -- Mu:' + ' ' + str(mu) + ' ' + 'Sigma:' + ' ' + str(sigma) )

In [ ]:
import multiprocessing
import numpy as np
multiprocessing.set_start_method('spawn', True)

NUM_CORES = 30

df_chunks_out_count = np.array_split(df_main, NUM_CORES)

with multiprocessing.Pool(NUM_CORES) as pool:

    df_main = pd.concat(pool.map(multi_df.nphi_filtering, df_chunks_out_count), ignore_index=True)

        * RHOB Variable

In [ ]:
(mu, sigma) = stats.norm.fit(df_main[df_main.RHOB.notnull()].RHOB.values) 

In [ ]:
plt.figure(figsize=(15,10))

sns.distplot(df_main[df_main.RHOB.notnull()].RHOB.values); 

plt.legend('Normal distribution')
plt.ylabel('Frequency')
plt.title('RHOB distribution -- Mu:' + ' ' + str(mu) + ' ' + 'Sigma:' + ' ' + str(sigma) )

        * GR Variable

In [ ]:
(mu, sigma) = stats.norm.fit(df_main[df_main.GR.notnull()].GR.values) 

In [ ]:
plt.figure(figsize=(15,10))

sns.distplot(df_main[df_main.GR.notnull()].GR.values); 

plt.legend('Normal distribution')
plt.ylabel('Frequency')
plt.title('GR distribution -- Mu:' + ' ' + str(mu) + ' ' + 'Sigma:' + ' ' + str(sigma) )

        * DTC Variable

In [ ]:
(mu, sigma) = stats.norm.fit(df_main[df_main.DTC.notnull()].DTC.values) 

In [ ]:
plt.figure(figsize=(15,10))

sns.distplot(df_main[df_main.DTC.notnull()].DTC.values); 

plt.legend('Normal distribution')
plt.ylabel('Frequency')
plt.title('DTC distribution -- Mu:' + ' ' + str(mu) + ' ' + 'Sigma:' + ' ' + str(sigma) )

        * RDEP Variable

In [ ]:
(mu, sigma) = stats.norm.fit(df_main[df_main.RDEP.notnull()].RDEP.values) 

In [ ]:
plt.figure(figsize=(15,10))

sns.distplot(df_main[df_main.RDEP.notnull()].RDEP.values); 

plt.legend('Normal distribution')
plt.ylabel('Frequency')
plt.title('RDEP distribution -- Mu:' + ' ' + str(mu) + ' ' + 'Sigma:' + ' ' + str(sigma) )

* Relationship with the categorical variable

        * CALI

In [ ]:
plt.figure(figsize=(15,10))
data = pd.concat([df_main[df_main.LITHOLOGY_GEOLINK.notnull()].LITHOLOGY_GEOLINK, df_main[df_main.LITHOLOGY_GEOLINK.notnull()].CALI], axis=1)
fig = sns.boxplot(x="LITHOLOGY_GEOLINK", y='CALI', data=data)


        * NPHI

In [ ]:
plt.figure(figsize=(15,10))
data = pd.concat([df_main[df_main.LITHOLOGY_GEOLINK.notnull()].LITHOLOGY_GEOLINK, df_main[df_main.LITHOLOGY_GEOLINK.notnull()].NPHI], axis=1)
fig = sns.boxplot(x="LITHOLOGY_GEOLINK", y='NPHI', data=data)


        * RHOB

In [ ]:
plt.figure(figsize=(15,10))
data = pd.concat([df_main[df_main.LITHOLOGY_GEOLINK.notnull()].LITHOLOGY_GEOLINK, df_main[df_main.LITHOLOGY_GEOLINK.notnull()].RHOB], axis=1)
fig = sns.boxplot(x="LITHOLOGY_GEOLINK", y='RHOB', data=data)


        * GR

In [ ]:
plt.figure(figsize=(15,10))
data = pd.concat([df_main[df_main.LITHOLOGY_GEOLINK.notnull()].LITHOLOGY_GEOLINK, df_main[df_main.LITHOLOGY_GEOLINK.notnull()].GR], axis=1)
fig = sns.boxplot(x="LITHOLOGY_GEOLINK", y='GR', data=data)


        * DTC

In [ ]:
plt.figure(figsize=(15,10))
data = pd.concat([df_main[df_main.LITHOLOGY_GEOLINK.notnull()].LITHOLOGY_GEOLINK, df_main[df_main.LITHOLOGY_GEOLINK.notnull()].DTC], axis=1)
fig = sns.boxplot(x="LITHOLOGY_GEOLINK", y='DTC', data=data)


        * RDEP

In [ ]:
plt.figure(figsize=(15,10))
data = pd.concat([df_main[df_main.LITHOLOGY_GEOLINK.notnull()].LITHOLOGY_GEOLINK, df_main[df_main.LITHOLOGY_GEOLINK.notnull()].RDEP], axis=1)
fig = sns.boxplot(x="LITHOLOGY_GEOLINK", y='RDEP', data=data)


* Outliers Classification

        * Number of outliers per row column

In [ ]:
val = 'Number of Outliers'

df_main[val] = 0

        * Non-Outlier Ranges (Expert Provided) (Used inside classify_outliers function! In here only for display purpose)

In [ ]:
ranges = {}

ranges['CALI'] = [0, 30]

ranges['NPHI'] = [0.1, 0.65] 

ranges['RHOB'] = [1, 4] 

ranges['GR'] = [0, 200]

ranges['DTC'] = [40, 200]

ranges['RDEP'] = [0.0001, 6000] 

        * Filling in the Numbers of Outliers Column

In [ ]:
import multiprocessing
import numpy as np
multiprocessing.set_start_method('spawn', True)

NUM_CORES = 7

df_chunks_out_count = np.array_split(df_main, NUM_CORES)

with multiprocessing.Pool(NUM_CORES) as pool:

    df_main = pd.concat(pool.map(multi_df.classify_outliers, df_chunks_out_count), ignore_index=True)

        * Dropping Outliers

In [ ]:
NUM_CORES = 7

df_chunks_out_count = np.array_split(df_main, NUM_CORES)

with multiprocessing.Pool(NUM_CORES) as pool:

    df_main = pd.concat(pool.map(multi_df.remove_outliers_class, df_chunks_out_count), ignore_index=True)

* Null Values Analysis

        * Raw Count

In [ ]:
columns = df_main.columns.to_list()

columns.remove('DEPTH')

columns.remove('WELL_NAME')

columns.remove('Number of Outliers')

In [ ]:
plt.figure(figsize=(15,10))

sns.barplot(x=df_main.isnull().sum().index, y=df_main.isnull().sum()) # only considering variables of interest
plt.xticks(rotation='90')
plt.xlabel('Variables', fontsize=10)
plt.ylabel('Total missing values (Not a Number)', fontsize=10) # Not counting -999,25
plt.title('Total missing values (Not a Number)', fontsize=15) # Not counting -999,25

        * Closer Look

In [ ]:
plt.figure(figsize=(15,10))

sns.barplot(x=df_main[columns].isnull().sum().index, y=df_main[columns].isnull().sum()) # only considering variables of interest
plt.xticks(rotation='90')
plt.xlabel('Variables', fontsize=10)
plt.ylabel('Total missing values (Not a Number)', fontsize=10) # Not counting -999,25
plt.title('Total missing values (Not a Number)', fontsize=15) # Not counting -999,25

        * Dropping Rows for Null values in Pivot Columns

In [ ]:
df_main = df_main.dropna(subset=['CALI', 'RHOB', 'GR', 'DTC', 'RDEP'])

In [ ]:
plt.figure(figsize=(15,10))

sns.barplot(x=df_main.isnull().sum().index, y=df_main.isnull().sum()) # only considering variables of interest
plt.xticks(rotation='90')
plt.xlabel('Variables', fontsize=10)
plt.ylabel('Total missing values (Not a Number)', fontsize=10) 
plt.title('Total missing values (Not a Number)', fontsize=15) 

## NPHI Prediction

* NPHI non NULL data

In [ ]:
nphi_data = df_main[pd.notnull(df_main['NPHI'])].drop(columns=['DEPTH', 'Number of Outliers', 'WELL_NAME'])

In [ ]:
nphi_data.head()

In [ ]:
len(nphi_data)

* Pearson Correlation Matrix

In [ ]:
corrmat_nphi = abs(nphi_data.corr()) # absolute correlation

plt.figure(figsize=(15,10))

sns.heatmap(corrmat_nphi, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, cmap='Blues')

        * Using the second branch, only the three highest correlated variables

In [ ]:
plt.figure(figsize=(10,10))

sns.scatterplot(x="NPHI", y="CALI", data=nphi_data.sample(100000))

# Linear correlation (maybe)

In [ ]:
plt.figure(figsize=(10,10))

sns.scatterplot(x="NPHI", y="RHOB", data=nphi_data.sample(100000))

# negative correlation

In [ ]:
plt.figure(figsize=(10,10))

sns.scatterplot(x="NPHI", y="DTC", data=nphi_data.sample(100000))

# non-linear correlation (exponential)

* Regression

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split, KFold, RandomizedSearchCV

from sklearn.preprocessing import RobustScaler

        * Dataset Split

In [ ]:
X = nphi_data[['CALI', 'DTC', 'RHOB']]

Y = nphi_data['NPHI'].values

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.40, random_state=42)

        * Model Selection

In [ ]:
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

                * Pipelines for Models (using Robust Scaler due possible presence of Outlier -- Decrease the sensitivity)

In [ ]:
linear_pipelines = []

linear_pipelines.append(('ScaledLASSO', Pipeline([('Scaler', RobustScaler()),('LASSO', linear_model.Lasso())])))
linear_pipelines.append(('ScaledEN', Pipeline([('Scaler', RobustScaler()),('EN', linear_model.ElasticNet())])))
linear_pipelines.append(('ScaledKNN', Pipeline([('Scaler', RobustScaler()),('KNN', KNeighborsRegressor())])))
linear_pipelines.append(('ScaledCART', Pipeline([('Scaler', RobustScaler()),('CART', DecisionTreeRegressor())])))
linear_pipelines.append(('ScaledGBM', Pipeline([('Scaler', RobustScaler()),('GBM', GradientBoostingRegressor())])))
linear_pipelines.append(('ScaledRidge', Pipeline([('Scaler', RobustScaler()),('Ridge', linear_model.Ridge())])))
linear_pipelines.append(('ScaledOMP', Pipeline([('Scaler', RobustScaler()),('OMP', linear_model.OrthogonalMatchingPursuit())])))
linear_pipelines.append(('ScaledBAYRID', Pipeline([('Scaler', RobustScaler()),('BAYRID', linear_model.BayesianRidge())])))
linear_pipelines.append(('ScaledSGD', Pipeline([('Scaler', RobustScaler()),('SGD', linear_model.SGDRegressor())])))
linear_pipelines.append(('ScaledRANDOMFOREST', Pipeline([('Scaler', RobustScaler()),('RANDOMFOREST', RandomForestRegressor(n_jobs=30))])))



                * Cross-Validation        

In [ ]:
results = []

names = []

for name, model in tqdm_notebook(linear_pipelines, desc='Cross-Validation Procedure'):

    kfold = KFold(n_splits=5, random_state=42)

    rmse = np.sqrt(-cross_val_score(model, x_train, y_train, cv=kfold, scoring='neg_mean_squared_error'))
    results.append(rmse)
    names.append(name)
    msg = "%s: %f (%f)" % (name, rmse.mean(), rmse.std())
    print(msg)

        * Hyperparemeter Tunning

            Due to this time-consuming task, proximately 5 hours using a 64 core server on multiprocessing, we will only present the best set of parameters. The result of the cell bellow can be verified by the files in the model's result folder. 

In [ ]:
# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]
# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}# Number of trees in random forest
# Use the random grid to search for best hyperparameters
# First create the base model to tune
#rf = RandomForestRegressor(n_jobs=20)

#kfold = KFold(n_splits=3, random_state=42)

#rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = kfold, verbose=10, random_state=42, n_jobs = 32, scoring='neg_mean_squared_error')

#rf_random.fit(RobustScaler().fit_transform(x_train), y_train)


        * Test Prediction

In [ ]:
best_rf = RandomForestRegressor(max_depth=30, max_features='sqrt', min_samples_split=5, n_estimators=400, n_jobs=7) # the rest of the best parameters are  the default ones

best_rf.fit(RobustScaler().fit_transform(x_train), y_train)

In [ ]:
y_predict = best_rf.predict(RobustScaler().fit_transform(x_test))

        * Metrics

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

print('MAE: ', str(mean_absolute_error(y_predict, y_test)), '\n')
print('########################', '\n')
print('RMSE: ', str(mean_squared_error(y_predict, y_test, squared=False)))

        * Null values Prediction

            * Null values Dataset

In [ ]:
null_nphi_dataset = df_main[pd.isnull(df_main['NPHI'])].drop(columns=['DEPTH', 'Number of Outliers', 'WELL_NAME'])

null_nphi_dataset.head(n=5)

            * Features to Predict (Highest correlation with target variable)

In [ ]:
features_for_predict = null_nphi_dataset[['CALI', 'DTC', 'RHOB']]

features_for_predict.head(n=5)

            * Prediction

In [ ]:
nphi_prediction = best_rf.predict(RobustScaler().fit_transform(features_for_predict))

            * Check boundary constraints

In [ ]:
print('Maximal Range Respected: ', nphi_prediction.max() < ranges['NPHI'][1], '\n', 'Minimal Range Respected: ', nphi_prediction.min() > ranges['NPHI'][0]) # Check if the prediction is respecting the previous established interval

            * Replacing Null values for Predicted ones

In [ ]:
predicted_nphi_dataset = null_nphi_dataset

predicted_nphi_dataset['NPHI'] = nphi_prediction

predicted_nphi_dataset.head(n=5)

        * Final Dataframe

In [ ]:
total_df = nphi_data.append(predicted_nphi_dataset)

total_df.sort_index(inplace=True)

total_df['DEPTH'] = df_main['DEPTH'].values

total_df['WELL_NAME'] = df_main['WELL_NAME'].values

total_df.head(n=5)

# Checkpoint 1

In [ ]:
# path_file = '../Data/total.csv.gz'

# total_df.to_csv(path_file,index=False, compression='gzip')

* Importing libraries for checkpoint 1

In [ ]:
sys.path.append('../')

import lasio
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from collections import Counter
import multiprocessing

from Source.Utils import welllog
from Source.Utils import multi_df
from Source.Utils import well_plot

from tqdm import tqdm_notebook

pd.set_option('max_columns', None)

In [ ]:
total_df = pd.read_csv('../Data/total.csv.gz', compression='gzip')

In [ ]:
total_df.head()

## Well clustering

* Preparing the clustering dataset

In [ ]:
tmp_dict = {}

wells = total_df['WELL_NAME'].unique().tolist()

for well in tqdm_notebook(wells, desc='Process Progress'):

    GR = total_df[total_df['WELL_NAME'] == well]['GR'].values

    RHOB = total_df[total_df['WELL_NAME'] == well]['RHOB'].values

    NPHI = total_df[total_df['WELL_NAME'] == well]['NPHI'].values

    DTC = total_df[total_df['WELL_NAME'] == well]['DTC'].values

    RDEP = total_df[total_df['WELL_NAME'] == well]['RDEP'].values

    listafinal = np.concatenate((GR, RHOB, NPHI, DTC, RDEP))

    tmp_dict[well] = listafinal

df_clust = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in tmp_dict.items() ]))

* Filtering Wells with High percentage of Null values

In [ ]:
len(df_clust.columns)

In [ ]:
null_count = df_clust.isnull().sum()

tresh = int(np.percentile(null_count, 50))

df_clust_clean = df_clust.copy()

for well in tqdm_notebook(df_clust.columns, desc='Process Progress'):
    if null_count[well] > tresh:
        df_clust_clean.drop(columns=well, inplace=True)


* Filling Null values

In [ ]:
df_clust_clean.fillna(df_clust_clean.mean(), inplace=True)

In [ ]:
df_clust_clean.isnull().sum().max()

* Importing the geographic locations of the drilled wells

In [ ]:
well_exploration = pd.read_csv('../Data/wellbore_exploration_all.csv')

In [ ]:
well_exploration.head()

In [ ]:
geo_well = well_exploration[['wlbWellboreName','wlbNsUtm', 'wlbEwUtm']]

In [ ]:
geo_well.head()

* Transposing the dataframe to prepare for the clustering algorithm

In [ ]:
df_clust_t = df_clust_clean.T

* Filtering well Northing and Easting coordinates and appending to the clusterized dataframe

In [ ]:
list_of_wells = df_clust_clean.columns

In [ ]:
df_clust_t['UTM-N'] = 0.0
df_clust_t['UTM-E'] = 0.0

In [ ]:
for well_name in tqdm_notebook(list_of_wells, desc='Process Progress'):
    if well_name in list(geo_well['wlbWellboreName'].values):
        df_clust_t.loc[well_name, 'UTM-N'] = geo_well[geo_well['wlbWellboreName'] == well_name]['wlbNsUtm'].values
        df_clust_t.loc[well_name, 'UTM-E'] = geo_well[geo_well['wlbWellboreName'] == well_name]['wlbEwUtm'].values

In [ ]:
df_clust_t[['UTM-N', 'UTM-E']] 

In [ ]:
df_clust_t = df_clust_t[df_clust_t['UTM-N'] != 0]

* Normalizing columns 

In [ ]:
df_clust_norm = df_clust_t.copy()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

df_clust_norm = MinMaxScaler().fit_transform(df_clust_norm)

df_clust_norm = pd.DataFrame(df_clust_norm, index=df_clust_t.index, columns=df_clust_t.columns)

* Importing K-means algorithm

In [ ]:
from sklearn.cluster import KMeans

* Evaluating the optimum number of clusters

In [ ]:
wcss = [] # Within cluster sum of squares to analyze k-means performance

k_number_clusters = np.arange(1, 11) # define number of clusters to test

for k in tqdm_notebook(k_number_clusters, desc='K-Means Hyperparameter Tunning'): 

    kmeans = KMeans(n_clusters=k, init="k-means++", random_state=42, max_iter=500, n_jobs=9) # k-means model definition

    kmeans.fit(df_clust_t) # fitting to our dataframe

    wcss.append(kmeans.inertia_) # appending intertia value to our list for further evaluation

In [ ]:
plt.figure(figsize=(12,6))    
plt.plot(k_number_clusters, wcss, linewidth=2, color="red", marker ="8")
plt.xlabel("K Clusters Value")
plt.xticks(np.arange(1,11,1))
plt.ylabel("WCSS")
plt.title('K-Means Elbow Plot Evaluation Method')
plt.show()

In [ ]:
N_CLUSTERS = 4

In [ ]:
optimum_clustering = KMeans(n_clusters=N_CLUSTERS, init="k-means++", random_state=42, max_iter=500, n_jobs=9)

df_clust_t['Cluster'] = optimum_clustering.fit_predict(df_clust_t)

In [ ]:
plt.figure(figsize=(12,6))    

sns.scatterplot(data=df_clust_t, x="UTM-E", y="UTM-N", palette='bright', hue="Cluster")
